In [4]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [5]:
import warnings

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import os

In [6]:
warnings.filterwarnings('ignore')

In [7]:
# Initialize Spark session
spark = SparkSession.builder.appName("CrimePatrol").getOrCreate()

# Load dataset
crime_data = spark.read.csv("Major_Crime_Indicators_Open_Data.csv", header=True, inferSchema=True)

# Explore the dataset
crime_data.printSchema()


root
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- OBJECTID: integer (nullable = true)
 |-- EVENT_UNIQUE_ID: string (nullable = true)
 |-- REPORT_DATE: string (nullable = true)
 |-- OCC_DATE: string (nullable = true)
 |-- REPORT_YEAR: integer (nullable = true)
 |-- REPORT_MONTH: string (nullable = true)
 |-- REPORT_DAY: integer (nullable = true)
 |-- REPORT_DOY: integer (nullable = true)
 |-- REPORT_DOW: string (nullable = true)
 |-- REPORT_HOUR: integer (nullable = true)
 |-- OCC_YEAR: integer (nullable = true)
 |-- OCC_MONTH: string (nullable = true)
 |-- OCC_DAY: integer (nullable = true)
 |-- OCC_DOY: integer (nullable = true)
 |-- OCC_DOW: string (nullable = true)
 |-- OCC_HOUR: integer (nullable = true)
 |-- DIVISION: string (nullable = true)
 |-- LOCATION_TYPE: string (nullable = true)
 |-- PREMISES_TYPE: string (nullable = true)
 |-- UCR_CODE: integer (nullable = true)
 |-- UCR_EXT: integer (nullable = true)
 |-- OFFENCE: string (nullable = true)
 |-- M

In [11]:
#drop invalid entries

#drop entries with latitude or longitued of 0 whicha re invalid in our data
crime_data = crime_data.filter(crime_data["LAT_WGS84"] != 0)
crime_data = crime_data.filter(crime_data["LONG_WGS84"] != 0)

#drop entries that were inputted with an hour outside the 24 hr clock
crime_data = crime_data.filter((col("OCC_HOUR") >= 0) & (col("OCC_HOUR") <= 23))

#drop entries without a valid day of the week
valid_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
crime_data = crime_data.filter(trim(crime_data["OCC_DOW"]).isin(valid_days))


crime_data.show(15)